# Debugging

### Good tutorial on building TFRecord files


https://medium.com/@WuStangDan/step-by-step-tensorflow-object-detection-api-tutorial-part-2-converting-dataset-to-tfrecord-47f24be9248d

### Useful scripts for reading TFRecord

We can read the contents of the TFRecord file holding NoteSequence records like this:

The following is a more general record iterator, which accepts as a second argument the protocol buffer class to be used for deserialization. Yields a generator.

Frequently used protos classes include:
    * `tf.train.SequenceExample`
    * `music_pb2.NoteSequence`

### Helpful IO for debugging Pipelines [`1.2 Preprocess.ipynb`]

In [1]:
# First, load a `.tfrecord` of NoteSequences.
tfrecord_path = './data/note_seqs/inputs.tfrecord'

In [120]:
records = list()
for record in pipeline.tf_record_iterator(tfrecord_path, music_pb2.NoteSequence):
    records.append(record)

# Hint: To look at first record, run `records[0]`
# Hint 2: This is useful to drill inside: records[0].feature_lists.feature_list['inputs'].feature

##### Instantiate pipelines

In [121]:
from magenta.models.performance_rnn import performance_model
from magenta.pipelines import note_sequence_pipelines

config = constants.default_configs['performance']

quantizer_instance = note_sequence_pipelines.Quantizer(steps_per_second = config.steps_per_second,
                                                       name='Quantizer_jupyter')
perf_extractor_instance = PerformanceExtractor(num_velocity_bins = config.num_velocity_bins)
encoder_pipeline_instance = EncoderPipeline(config,
                                            name='EncoderPipeline_jupyter')

##### Step-through pipelines

In [198]:
for record in pipeline.tf_record_iterator(tfrecord_path, music_pb2.NoteSequence):
    note_sequence1 = record
# note_sequence1.SerializeToString()
# note_sequence1

In [180]:
note_sequence2 = quantizer_instance.transform(note_sequence1)[0]

In [563]:
note_sequence3 = perf_extractor_instance.transform(note_sequence2)[0]

# note_sequence3.to_sequence()        # converts Performance to NoteSequence proto
# note_sequence3.__getitem__(4)       # return event at position
# note_sequence3.steps_per_second     # if Performance(BasePerformance)
# note_sequence3.steps_per_quarter    # if MetricPerformance(BasePerformance)
# note_sequence3._events              # list of all events
# len(note_sequence3._events)         # len of events
# note_sequence3.num_steps            # len of sequence in quantized steps
# note_sequence3.steps                # list of the time step at each event in the sequence

# returns an iterator
# for i, event in enumerate(note_sequence3.__iter__()): 
#     print(event)
#     if i > 25:
#         break

In [261]:
note_sequence4 = encoder_pipeline_instance.transform(note_sequence3)[0]
# note_sequence4